In [109]:
import pandas as pd
from collections import Counter
import itertools
import copy

In [7]:
###2
## Gale-Shapley Algorithm best to recipients:
def gale_shapley_1(don_pre, reci_pre):
    don_available = {recipient: donors_names[:] for recipient in recipients_names}
    waiting_list = []
    proposals = {}
    
    while len(waiting_list) < len(recipients_names):
        for recipient in recipients_names:
            if recipient not in waiting_list:
                donor = don_available[recipient]
                best_choice = reci_pre.loc[recipient][reci_pre.loc[recipient].index.isin(donor)].idxmin()
                proposals[(recipient, best_choice)] = (reci_pre.loc[recipient][best_choice], don_pre.loc[best_choice][recipient])
        
        overlays = Counter([key[1] for key in proposals.keys()])
        
        for donor in overlays.keys():
            if overlays[donor] > 1:
                pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() if donor in pair}.items(), key=lambda x: x[1][1])[1:]
                for p_to_drop in pairs_to_drop:
                    del proposals[p_to_drop[0]]
                    _donor = copy.copy(don_available[p_to_drop[0][0]])
                    _donor.remove(p_to_drop[0][1])
                    don_available[p_to_drop[0][0]] = _donor
        
        waiting_list = [recipient[0] for recipient in proposals.keys()]
    
    return proposals

## Gale-Shapley Algorithm reversed:
def gale_shapley_2(reci_pre, don_pre):
    reci_available = {donor: recipients_names[:] for donor in donors_names}
    waiting_list = []
    proposals = {}
    
    while len(waiting_list) < len(donors_names):
        for donor in donors_names:
            if donor not in waiting_list:
                recipient = reci_available[donor]
                best_choice = don_pre.loc[donor][don_pre.loc[donor].index.isin(recipient)].idxmin()
                proposals[(donor, best_choice)] = (don_pre.loc[donor][best_choice], reci_pre.loc[best_choice][donor])
        
        overlays = Counter([key[1] for key in proposals.keys()])
        
        for recipient in overlays.keys():
            if overlays[recipient] > 1:
                pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() if recipient in pair}.items(), key=lambda x: x[1][1])[1:]
                for p_to_drop in pairs_to_drop:
                    del proposals[p_to_drop[0]]
                    _recipient = copy.copy(reci_available[p_to_drop[0][0]])
                    _recipient.remove(p_to_drop[0][1])
                    reci_available[p_to_drop[0][0]] = _recipient
        
        waiting_list = [donor[0] for donor in proposals.keys()]
    
    return proposals

###3
# 检查互斥性并移除不可能在任何稳定匹配中配对的项
def check_and_remove_exclusivity(filtered_reci_pre, filtered_don_pre):
    # 移除不在对方偏好列表中的 donor
    for r, donors in filtered_reci_pre.items():
        for d in donors[:]:  # 使用切片复制列表，避免修改时影响遍历
            if r not in filtered_don_pre[d]:
                filtered_reci_pre[r].remove(d)

    # 移除不在对方偏好列表中的 recipient
    for d, recipients in filtered_don_pre.items():
        for r in recipients[:]:  # 使用切片复制列表，避免修改时影响遍历
            if d not in filtered_reci_pre[r]:
                filtered_don_pre[d].remove(r)

    return filtered_reci_pre, filtered_don_pre

# 按照矩阵的顺序重新排列偏好列表的函数
def reorder_preference(pref_dict, pref_matrix):
    reordered_dict = {}
    for key, values in pref_dict.items():
        reordered_dict[key] = sorted(values, key=lambda x: pref_matrix.loc[key, x])
    return reordered_dict

###4
def find_closed_cycles(filtered_reci_pre, max_length=10):
    # Create empty lists to store cycles and closed cycles
    cycle = []
    closed_cycle = []

    # First step: Create initial lists for recipients with preference lists longer than 1
    for r in filtered_reci_pre:
        if len(filtered_reci_pre[r]) > 1:
            # Initialize the list with the recipient and their second preferred donor
            initial_list = [r, filtered_reci_pre[r][1]]
            cycle.append(initial_list)
    # print(cycle)

    # Loop until cycle is empty
    while cycle:
        new_cycle = []  # To store new lists generated in each iteration

        # Check each list in cycle
        for lst in cycle[:]:  # Use a copy of cycle to modify the original
            # First, check if the list forms a closed cycle
            if lst[0] == lst[-1]:
                closed_cycle.append(lst)  # Store the closed cycle
                cycle.remove(lst)  # Remove from cycle
            elif len(lst) > max_length:
                # Remove lists that exceed maximum allowed length
                cycle.remove(lst)
            elif len(set(lst)) < len(lst):
                # Remove lists with repeated recipients/donors (excluding the case where start and end are the same)
                cycle.remove(lst)
            else:
                if len(lst) % 2 == 1:  # If the list length is odd
                    last_recip = lst[-1]
                    if len(filtered_reci_pre[last_recip]) > 1:
                        next_donor = filtered_reci_pre[last_recip][1]  # Take the second preference
                        lst.append(next_donor)  # Extend the list
                elif len(lst) % 2 == 0:  # If the list length is even
                    last_donor = lst[-1]
                    found = False
                    # Find recipients whose first preference matches the last donor
                    for recip, preferences in filtered_reci_pre.items():
                        if preferences[0] == last_donor:
                            new_list = lst + [recip]
                            new_cycle.append(new_list)
                            found = True
                    cycle.remove(lst)  # Remove the original list
                    if not found:
                        # If no matching recipient found, do nothing (lst already removed)
                        pass

        # Add new lists to cycle
        cycle.extend(new_cycle)

    return closed_cycle

###5
def update_matching(mu_R, output_lists, reci_pre_matrix, don_pre_matrix):
    # 创建list来存储所有新的匹配
    matchings = []
    
    # 逐个遍历output_lists中的list
    for lst in output_lists:
        # 复制当前的匹配，准备更新
        new_mu_R = mu_R.copy()
        
        # 逐个解析list，将r和d成对匹配
        for i in range(0, len(lst) - 1, 2):
            r = lst[i]
            d = lst[i + 1]
            
            # 获取r对d的偏好序数
            r_preference = reci_pre_matrix.loc[r, d]
            
            # 获取d对r的偏好序数
            d_preference = don_pre_matrix.loc[d, r]
            
            # 生成新的配对，并替换原mu_R中包含这个r的配对
            new_mu_R[(r, d)] = (r_preference, d_preference)
            
            # 删除旧的包含r的配对
            for key in list(new_mu_R.keys()):
                if key[0] == r and key[1] != d:
                    del new_mu_R[key]
        
        # 将新的匹配加入到matchings列表中
        matchings.append(new_mu_R.copy())
    
    return matchings

###6
def combine_matchings(matchings, reci_pre_matrix):
    # Initialize the new matching
    combined_matching = {}

    # Get the list of all recipients
    recipients = reci_pre_matrix.index

    # For each recipient, select a donor from the combined matchings
    for recipient in recipients:
        worst_donor = None
        worst_preference = -1  # Higher preference value means the recipient likes the donor less
        matching_with_worst_donor = None  # To store the matching containing the worst donor

        # Iterate over all matchings
        for matching in matchings:
            for (r, d), (r_pref, _) in matching.items():
                if r == recipient and r_pref > worst_preference:
                    worst_preference = r_pref
                    worst_donor = (r, d)
                    matching_with_worst_donor = matching  # Record the matching containing this pair

        # Check if a worst donor was found and add it to the combined matching
        if worst_donor:
            combined_matching[worst_donor] = (worst_preference, matching_with_worst_donor[worst_donor][1])
        else:
            print(f"No valid donor found for recipient {recipient}")

    return combined_matching


def generate_pairwise_combinations_and_combine(matchings, reci_pre_matrix):
    all_combined_matchings = []

    # Generate all possible pairwise combinations of matchings
    for combination in itertools.combinations(matchings, 2):
        print(f"Combining 2 matchings: {combination}")
        # Combine the two matchings into a new matching
        new_matching = combine_matchings(combination, reci_pre_matrix)
        if new_matching:
            all_combined_matchings.append(new_matching)

    return all_combined_matchings

def add_unique_matchings(all_combined_matchings, stable_matching):
    # Convert each matching in stable_matching to a frozenset to compare
    stable_set = {frozenset(matching.items()) for matching in stable_matching}

    # Iterate over all_combined_matchings and check for duplicates
    for matching in all_combined_matchings:
        matching_frozenset = frozenset(matching.items())  # Convert current matching to frozenset
        if matching_frozenset not in stable_set:
            stable_matching.append(matching)  # Add the non-duplicate matching
            stable_set.add(matching_frozenset)  # Update the stable_set

###7
def update_filtered_preference_lists(final_matching, filtered_reci_pre, filtered_don_pre, reci_pre_matrix, don_pre_matrix):
    # Update recipients' preference lists
    for (recipient, donor), (r_pref, _) in final_matching.items():
        # Get the recipient's current list of preferred donors
        filtered_donors = filtered_reci_pre[recipient]
        # Remove donors that the recipient prefers more than the matched donor
        filtered_reci_pre[recipient] = [d for d in filtered_donors if reci_pre_matrix.loc[recipient, d] >= r_pref]

    # Update donors' preference lists
    for (recipient, donor), (_, d_pref) in final_matching.items():
        # Get the donor's current list of preferred recipients
        filtered_recipients = filtered_don_pre[donor]
        # Remove recipients that the donor prefers less than the matched recipient
        filtered_don_pre[donor] = [r for r in filtered_recipients if don_pre_matrix.loc[donor, r] <= d_pref]

    return filtered_reci_pre, filtered_don_pre

def check_and_remove_exclusivity(filtered_reci_pre, filtered_don_pre):
    # Remove donors not in the donor's preference list
    for r, donors in filtered_reci_pre.items():
        for d in donors[:]:  # Use a copy to avoid modifying the list while iterating
            if r not in filtered_don_pre[d]:
                filtered_reci_pre[r].remove(d)

    # Remove recipients not in the recipient's preference list
    for d, recipients in filtered_don_pre.items():
        for r in recipients[:]:  # Use a copy to avoid modifying the list while iterating
            if d not in filtered_reci_pre[r]:
                filtered_don_pre[d].remove(r)

    return filtered_reci_pre, filtered_don_pre

# 将字典转换为frozenset来检测重复，并去除重复项
def remove_duplicate_dicts(all_new_matchings):
    unique_matchings = []
    seen = set()  # 用来记录已经出现过的匹配

    for matching in all_new_matchings:
        # 将字典的项转化为frozenset，从而能够进行hash并比较
        matching_frozenset = frozenset(matching.items())
        
        # 如果该匹配没有出现过，则将其加入unique_matchings
        if matching_frozenset not in seen:
            unique_matchings.append(matching)
            seen.add(matching_frozenset)

    return unique_matchings

In [38]:
# Step 1: 定义 recipients 和 donors 的偏好列表

# Define the recipient preference dictionary
reci_pref = {
    'r1': ['d3', 'd1', 'd5', 'd7', 'd4', 'd2', 'd8', 'd6'],
    'r2': ['d6', 'd1', 'd3', 'd4', 'd8', 'd7', 'd5', 'd2'],
    'r3': ['d7', 'd4', 'd3', 'd6', 'd5', 'd1', 'd2', 'd8'],
    'r4': ['d5', 'd3', 'd8', 'd2', 'd6', 'd1', 'd4', 'd7'],
    'r5': ['d4', 'd1', 'd2', 'd8', 'd7', 'd3', 'd6', 'd5'],
    'r6': ['d6', 'd2', 'd5', 'd7', 'd8', 'd4', 'd3', 'd1'],
    'r7': ['d7', 'd8', 'd1', 'd6', 'd2', 'd3', 'd4', 'd5'],
    'r8': ['d2', 'd6', 'd7', 'd1', 'd8', 'd3', 'd4', 'd5']
}


# Define the donor preference dictionary
don_pref = {
    'd1': ['r4', 'r3', 'r8', 'r1', 'r2', 'r5', 'r7', 'r6'],
    'd2': ['r3', 'r7', 'r5', 'r8', 'r6', 'r4', 'r1', 'r2'],
    'd3': ['r7', 'r5', 'r8', 'r3', 'r6', 'r2', 'r1', 'r4'],
    'd4': ['r6', 'r4', 'r2', 'r7', 'r3', 'r1', 'r5', 'r8'],
    'd5': ['r8', 'r7', 'r1', 'r5', 'r6', 'r4', 'r3', 'r2'],
    'd6': ['r5', 'r4', 'r7', 'r6', 'r2', 'r8', 'r3', 'r1'],
    'd7': ['r1', 'r4', 'r5', 'r6', 'r2', 'r8', 'r3', 'r7'],
    'd8': ['r2', 'r5', 'r4', 'r3', 'r7', 'r8', 'r1', 'r6']
}

# Create the recipient preference matrix
reci_pref_matrix = pd.DataFrame(index=reci_pref.keys(), columns=['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8'])
for reci, prefs in reci_pref.items():
    for rank, donor in enumerate(prefs):
        reci_pref_matrix.loc[reci, donor] = rank + 1

# Create the donor preference matrix
don_pref_matrix = pd.DataFrame(index=don_pref.keys(), columns=['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8'])
for donor, prefs in don_pref.items():
    for rank, reci in enumerate(prefs):
        don_pref_matrix.loc[donor, reci] = rank + 1

# Display the matrices
print("Recipient Preference Matrix:")
print(reci_pref_matrix)
print("\nDonor Preference Matrix:")
print(don_pref_matrix)


Recipient Preference Matrix:
   d1 d2 d3 d4 d5 d6 d7 d8
r1  2  6  1  5  3  8  4  7
r2  2  8  3  4  7  1  6  5
r3  6  7  3  2  5  4  1  8
r4  6  4  2  7  1  5  8  3
r5  2  3  6  1  8  7  5  4
r6  8  2  7  6  3  1  4  5
r7  3  5  6  7  8  4  1  2
r8  4  1  6  7  8  2  3  5

Donor Preference Matrix:
   r1 r2 r3 r4 r5 r6 r7 r8
d1  4  5  2  1  6  8  7  3
d2  7  8  1  6  3  5  2  4
d3  7  6  4  8  2  5  1  3
d4  6  3  5  2  7  1  4  8
d5  3  8  7  6  4  5  2  1
d6  8  5  7  2  1  4  3  6
d7  1  5  7  2  3  4  8  6
d8  7  1  4  3  2  8  5  6


In [254]:
import numpy as np
import pandas as pd
import random

def generate_random_permutation(n):
    return np.random.permutation(n)

random.seed(0)
# 示例数据
donors_names = ['d' + str(i) for i in range(1, 10)]
recipients_names = ['r' + str(i) for i in range(1, 10)]

# 随机打乱donors和recipients的名字
np.random.shuffle(donors_names)
np.random.shuffle(recipients_names)

# 创建初始匹配
initial_match = list(zip(donors_names, recipients_names))

# 构造don_pre和reci_pre的示例数据
# 生成 reci_pre 数据框
reci_pre_arry = np.array([generate_random_permutation(9) for _ in range(9)])
reci_pref_matrix = pd.DataFrame(reci_pre_arry, index=recipients_names, columns=donors_names)

# 生成 don_pre 数据框
don_pre_arry = np.array([generate_random_permutation(9) for _ in range(9)])
don_pref_matrix = pd.DataFrame(don_pre_arry, index=donors_names, columns=recipients_names)



In [255]:
# Step 2: Gale-Shapley 算法实现

stable_matching = []

recipients_names = reci_pref_matrix.index.tolist()
donors_names = don_pref_matrix.index.tolist()

# 找到recipients最优匹配
mu_R2 = gale_shapley_1(don_pref_matrix, reci_pref_matrix)
# 找到donors最优匹配
mu_D2 = gale_shapley_2(reci_pref_matrix, don_pref_matrix)
stable_matching.append(mu_R2)

print("recipients最优匹配:", mu_R2)
print("donors最优匹配:", mu_D2)


recipients最优匹配: {('r3', 'd5'): (0, 1), ('r6', 'd3'): (0, 0), ('r2', 'd9'): (0, 3), ('r4', 'd4'): (0, 0), ('r7', 'd2'): (1, 0), ('r1', 'd7'): (3, 3), ('r9', 'd8'): (5, 2), ('r8', 'd6'): (3, 1), ('r5', 'd1'): (5, 5)}
donors最优匹配: {('d3', 'r6'): (0, 0), ('d4', 'r4'): (0, 0), ('d2', 'r7'): (0, 1), ('d6', 'r8'): (1, 3), ('d5', 'r3'): (1, 0), ('d9', 'r2'): (3, 0), ('d7', 'r1'): (3, 3), ('d8', 'r9'): (2, 5), ('d1', 'r5'): (5, 5)}


In [256]:
# Step 3: 根据 mu_R 和 mu_D 生成更新的偏好序列
# 对于每个 recipient r 筛选符合条件的 donors
filtered_reci_pre = {}
for r, d in mu_R2.keys():

    # 获取 r 对 μR 中匹配 donor 的偏好序数
    r_pref_muR = reci_pref_matrix.loc[r, d]
    
    # 获取 r 在 μD 中匹配的 donor
    matching_donor_in_muD = [key[0] for key in mu_D2.keys() if key[1] == r][0]
    r_pref_muD = reci_pref_matrix.loc[r, matching_donor_in_muD]

    # 筛选符合条件的 donors
    filtered_donors = reci_pref_matrix.columns[(reci_pref_matrix.loc[r] < r_pref_muR) | (reci_pref_matrix.loc[r] > r_pref_muD)].tolist()
    filtered_reci_pre[r] = filtered_donors

    # 删除不符合条件的 donors
    filtered_reci_pre[r] = reci_pref_matrix.loc[r].drop(filtered_donors).index.tolist()

# 对于每个 donor d 筛选符合条件的 recipients
filtered_don_pre = {}
for d, r in mu_D2.keys():
    
    # 获取 d 对 μD 中匹配 recipient 的偏好序数
    d_pref_muD = don_pref_matrix.loc[d, r]
    
    # 获取 d 在 μR 中匹配的 recipient
    matching_recipient_in_muR = [key[0] for key in mu_R2.keys() if key[1] == d][0]
    d_pref_muR = don_pref_matrix.loc[d, matching_recipient_in_muR]

    # 筛选符合条件的 recipients-
    filtered_recipients = don_pref_matrix.columns[(don_pref_matrix.loc[d] < d_pref_muD) | (don_pref_matrix.loc[d] > d_pref_muR)].tolist()
    filtered_don_pre[d] = filtered_recipients

    # 删除不符合条件的 recipients
    filtered_don_pre[d] = don_pref_matrix.loc[d].drop(filtered_recipients).index.tolist()


# 调用函数检查互斥性并更新简化偏好列表
filtered_reci_pre, filtered_don_pre = check_and_remove_exclusivity(filtered_reci_pre, filtered_don_pre)


# 重新排列后的 recipient 偏好顺序
filtered_reci_pre2 = reorder_preference(filtered_reci_pre, reci_pref_matrix)
filtered_don_pre2 = reorder_preference(filtered_don_pre, don_pref_matrix)
print(filtered_reci_pre2)
print(filtered_don_pre2)

{'r3': ['d5'], 'r6': ['d3'], 'r2': ['d9'], 'r4': ['d4'], 'r7': ['d2'], 'r1': ['d7'], 'r9': ['d8'], 'r8': ['d6'], 'r5': ['d1']}
{'d3': ['r6'], 'd4': ['r4'], 'd2': ['r7'], 'd6': ['r8'], 'd5': ['r3'], 'd9': ['r2'], 'd7': ['r1'], 'd8': ['r9'], 'd1': ['r5']}


In [257]:
# Step4：根据更新的偏好列表filtered_reci_pre1找出所有闭合的循环

closed_cycles2 = find_closed_cycles(filtered_reci_pre2)

# Remove duplicate cycles
def remove_duplicates(lists):
    seen = set()
    result = []

    for lst in lists:
        # Convert the list to a tuple of tuples to make it hashable
        lst_tuple = tuple(lst)
        if lst_tuple not in seen:
            result.append(lst)
            seen.add(lst_tuple)

    return result

output_lists = remove_duplicates(closed_cycles2)
# print(output_lists)

def remove_duplicate_cycles(cycles):
    unique_cycles = {}
    for cycle in cycles:
        # Convert the cycle to a frozenset to identify unique sets
        cycle_set = frozenset(cycle)
        if cycle_set not in unique_cycles:
            # Store the original cycle corresponding to the unique set
            unique_cycles[cycle_set] = cycle
    # Return the original cycles corresponding to the unique sets
    return list(unique_cycles.values())

output_lists = remove_duplicate_cycles(output_lists)
print(output_lists)

[]


In [258]:
# Step 5: 根据每一个闭合循环生成新的一个稳定匹配

new_matching = update_matching(mu_R2, output_lists, reci_pref_matrix, don_pref_matrix)
for i in new_matching:
    stable_matching.append(i)

print(new_matching)

[]


In [259]:
# # Step 6: 循环步骤 6(i)-6(iii) 直到找到与 mu_D 相同的稳定匹配

iterations = 0
while new_matching:
    iterations += 1
    print(f"\nIteration {iterations}:")
          
    # Step 6(i): 组合多个匹配生成新的稳定匹配
    # Assuming new_matchings and reci_pref_matrix are already defined
    # Generate all new pairwise matching combinations
    new_matchings = []
    new_matchings = generate_pairwise_combinations_and_combine(new_matching, reci_pref_matrix)

    # Add unique matchings to stable_matching
    add_unique_matchings(new_matchings, stable_matching)

    # Step 6(ii): 用最新的稳定匹配更新 recipients 和 donors 的偏好列表
    # Initialize lists to store the updated preference lists after each update
    updated_filtered_reci_pre_list = []
    updated_filtered_don_pre_list = []
    all_new_matchings = []

    # Loop over each matching in new_matchings
    for idx, final_matching in enumerate(new_matchings):
        print(f"Processing matching {idx + 1}:")
        
        # Make deep copies of the initial preference lists to avoid modifying the originals
        reci_pre_copy = copy.deepcopy(filtered_reci_pre2)
        don_pre_copy = copy.deepcopy(filtered_don_pre2)

        # Update the preference lists using the current matching
        updated_reci_pre, updated_don_pre = update_filtered_preference_lists(
            final_matching, reci_pre_copy, don_pre_copy, reci_pref_matrix, don_pref_matrix
        )

        # Check mutual exclusivity and remove impossible pairs
        updated_reci_pre, updated_don_pre = check_and_remove_exclusivity(updated_reci_pre, updated_don_pre)

        closed_cycles = find_closed_cycles(updated_reci_pre)
        output_lists = remove_duplicates(closed_cycles)
        output_lists = remove_duplicate_cycles(output_lists)

        # 如果没有闭合循环，则结束
        if not output_lists:
            print("No more closed cycles found. Terminating the loop.")
            break

        per_new_matchings = update_matching(final_matching, output_lists, reci_pref_matrix, don_pref_matrix)
        for m in per_new_matchings:
            all_new_matchings.append(m)
        
        # Store the updated preference lists
        updated_filtered_reci_pre_list.append(updated_reci_pre)
        updated_filtered_don_pre_list.append(updated_don_pre)

        # Optionally, print the updated preference lists
        print(f"New matchings after matching {idx + 1}:")
        print(per_new_matchings)
        # print(f"Updated Donors Preference List after matching {idx + 1}:")
        # print(updated_don_pre)
        print("")  # Empty line for better readability

    # 去除重复的字典
    new_matchings = []
    new_matchings = remove_duplicate_dicts(all_new_matchings)
    for i in new_matchings:
        stable_matching.append(i)

    # Step 6(iii): 用new_matchings的每一项来分别更新recipients和donors的偏好列表filtered_reci_pre和filtered_don_pre
    #         并根据闭合循环生成新的稳定匹配

    # Initialize lists to store the updated preference lists after each update
    updated_filtered_reci_pre_list = []
    updated_filtered_don_pre_list = []
    all_new_matchings = []

    # Loop over each matching in new_matchings
    for idx, final_matching in enumerate(new_matchings):
        print(f"Processing matching {idx + 1}:")
        
        # Make deep copies of the initial preference lists to avoid modifying the originals
        reci_pre_copy = copy.deepcopy(filtered_reci_pre2)
        don_pre_copy = copy.deepcopy(filtered_don_pre2)

        # Update the preference lists using the current matching
        updated_reci_pre, updated_don_pre = update_filtered_preference_lists(
            final_matching, reci_pre_copy, don_pre_copy, reci_pref_matrix, don_pref_matrix
        )

        # Check mutual exclusivity and remove impossible pairs
        updated_reci_pre, updated_don_pre = check_and_remove_exclusivity(updated_reci_pre, updated_don_pre)

        closed_cycles = find_closed_cycles(updated_reci_pre)
        output_lists = remove_duplicates(closed_cycles)
        output_lists = remove_duplicate_cycles(output_lists)
        per_new_matchings = update_matching(final_matching, output_lists, reci_pref_matrix, don_pref_matrix)
        for m in per_new_matchings:
            all_new_matchings.append(m)
        
        # Store the updated preference lists
        updated_filtered_reci_pre_list.append(updated_reci_pre)
        updated_filtered_don_pre_list.append(updated_don_pre)

        # Optionally, print the updated preference lists
        print(f"New matchings after matching {idx + 1}:")
        print(per_new_matchings)
        # print(f"Updated Donors Preference List after matching {idx + 1}:")
        # print(updated_don_pre)
        print("")  # Empty line for better readability

    # 去除重复的字典
    new_matching = []
    new_matching = remove_duplicate_dicts(all_new_matchings)
    for i in new_matching:
        stable_matching.append(i)


In [260]:
def update_stable_matching(stable_matching, mu_D2):
    # 标记mu_D2是否已经存在于stable_matching中的任意字典
    is_duplicate = False

    # 遍历stable_matching中的每个字典
    for match in stable_matching:
        # 遍历mu_D2中的每个配对
        for donor_recipient, value in mu_D2.items():
            # 反转mu_D2的key (donor, recipient) 以匹配stable_matching的(recipient, donor)
            recipient_donor = (donor_recipient[1], donor_recipient[0])
            
            # 如果发现匹配的recipient-donor对，标记为重复
            if recipient_donor in match:
                is_duplicate = True
                break
        if is_duplicate:
            break
    
    # 如果没有重复的mu_D2，添加mu_D2到stable_matching中
    if not is_duplicate:
        stable_matching.append(mu_D2)
    else:
        print(f'Donor optimal stable matching, mu_D2, exists in stable_matching.')

    return stable_matching

stable_matching = update_stable_matching(stable_matching, mu_D2)
print(len(stable_matching))


Donor optimal stable matching, mu_D2, exists in stable_matching.
1
